Import

In [ ]:
import pandas as pd
import numpy as np
import faiss
import pickle
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os , sys
sys.path.append(os.path.abspath('../scripts'))

from preprocessing import clean_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Teddy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Load Data

In [2]:
df = pd.read_csv('../data/complaints.csv')

df.head()

C:\Users\Teddy\AppData\Local\Temp\ipykernel_4324\1989515440.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/complaints.csv')


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,Experian Information Solutions Inc.,FL,32092,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195687
1,2025-06-20,Debt collection,Telecommunications debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company can't verify or dispute the facts in t...,"Eastern Account Systems of Connecticut, Inc.",FL,342XX,NaN,NaN,Web,2025-06-20,Closed with explanation,Yes,NaN,14195688
2,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AZ,85225,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195689
3,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,Experian Information Solutions Inc.,AZ,85225,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195690
4,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,Experian Information Solutions Inc.,IL,60628,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195692


Process and save

In [ ]:
#  Rename column for simplicity
df = df.rename(columns={'Consumer complaint narrative': 'complaint'})

# Handle Missing Values
df = df.dropna(subset=['complaint'])

# Clean Text
df['cleaned_complaint'] = df['complaint'].apply(clean_text)

# Save cleaned data
df.to_csv('../data/complaints_cleaned.csv', index=False)

# Preview cleaned sample
print(df[['complaint', 'cleaned_complaint']].head())


In [ ]:
%pip install langchain

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Text Chunking
chunk_size = 300
chunk_overlap = 50

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]
)

documents = []
for _, row in df.iterrows():
    chunks = splitter.split_text(row['cleaned_complaint'])
    for chunk in chunks:
        documents.append({
            "text": chunk,
            "metadata": {
                "complaint_id": row['Complaint ID'],
                "product": row['Product']
            }
        })


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached orjson-3.10.18-cp313-cp313-win_amd64.whl.metadata (43 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 4.8 MB/s eta 0:00:01
   ----------------------------------

In [ ]:
# Embedding with Sentence-Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')
texts = [doc['text'] for doc in documents]
embeddings = model.encode(texts, show_progress_bar=True)
embedding_matrix = np.array(embeddings)

In [ ]:
# FAISS Indexing
dimension = embedding_matrix.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embedding_matrix)

# Persist vector store
os.makedirs("vector_store", exist_ok=True)
faiss.write_index(index, "vector_store/faiss_index.index")

with open("vector_store/metadata.pkl", "wb") as f:
    pickle.dump([doc['metadata'] for doc in documents], f)

print("✅ Vector store and metadata saved to vector_store/")